In [1]:
import pandas as pd
import numpy as np

In [2]:
RW=pd.read_csv('rwanda_data.csv')


In [3]:
RW.head()

,rwanda_ntl,rwanda_gmap
0,0.34,0
1,0.22,0
2,0.22,0
3,0.26,0
4,0.27,0


In [4]:
RW_ntl=RW.rwanda_ntl.to_numpy()
RW_ntl=RW_ntl.reshape(-1,1)

In [5]:
from sklearn.mixture import GaussianMixture 
gmm = GaussianMixture(n_components=3,covariance_type='full')
gmm.fit(RW_ntl)

#predictions from gmm
labels = gmm.predict(RW_ntl)
frame = pd.DataFrame(RW_ntl)
frame['cluster'] = labels
frame.columns = ['RW_ntl','cluster']


In [6]:
frame[frame.cluster==0].describe()

,RW_ntl,cluster
count,110365.000000,110365.0
mean,0.230669,0.0
std,0.078996,0.0
min,0.010000,0.0
25%,0.180000,0.0
50%,0.220000,0.0
75%,0.280000,0.0
max,0.460000,0.0


In [7]:
frame[frame.cluster==1].describe()

,RW_ntl,cluster
count,5932.000000,5932.0
mean,0.759267,1.0
std,0.312860,0.0
min,0.470000,1.0
25%,0.520000,1.0
50%,0.630000,1.0
75%,0.910000,1.0
max,1.750000,1.0


In [8]:
frame[frame.cluster==2].describe()

,RW_ntl,cluster
count,2067.000000,2067.0
mean,6.880464,2.0
std,7.204469,0.0
min,1.760000,2.0
25%,2.520000,2.0
50%,4.120000,2.0
75%,8.415000,2.0
max,64.220000,2.0


In [9]:
RW['ntl'] = np.where(RW['rwanda_ntl']<1.76,'Medium','High')

In [10]:
RW.loc[(RW['rwanda_ntl'] <0.47),'ntl']='Low'

In [11]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [12]:
confusion_matrix = pd.crosstab(RW["ntl"], RW["rwanda_gmap"]).as_matrix()

In [13]:
stat, p, dof, expected = chi2_contingency(confusion_matrix)

In [14]:
prob=0.05
alpha = 1.0 - prob
if p <= alpha:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')

Dependent (reject H0)


In [15]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import seaborn as sns



def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))
cramers_v(confusion_matrix)

0.2992447011842873